#Получаем данные из датасета#

In [1]:
# brian2 is a simulator for spiking neural networks. It allows to simulate neuraldinamics
# using custom differential equations, but it is very computationally intensive.
# https://brian2.readthedocs.io/en/stable/resources/tutorials/index.html

!pip install brian2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import random

import numpy as np
import random

def load_data():
    data = np.genfromtxt('/content/dataset.csv', delimiter='\t', dtype=None, encoding=None)

    filtered_data = []
    for i in range(0, len(data), 14):
        arrays = []
        arrays.append(data[i:i+13])
        arrays = np.array(arrays)
        filtered_data.append(arrays)

    #filtered_array = np.array(filtered_data)
    filtered_array = np.squeeze(np.array(filtered_data), axis=1)

    n1 = len(filtered_data)
    data_size = int(0.8 * n1)

    x_train = filtered_array[:data_size]
    x_test = filtered_array[data_size:]

    filtered_labels = [row for index, row in enumerate(data) if (index + 1) % 14 == 0]
    filtered_labels_array = np.array(filtered_labels)
    n2 = len(filtered_labels_array)
    labels_size = int(0.8 * n2)
    y = []
    for i in range(len(filtered_labels_array)):
        y.append(filtered_labels_array[i][0])
    y = np.array(y)
    y_train = y[:labels_size]
    y_test = y[labels_size:]

    data_train = (x_train, y_train)
    data_test = (x_test, y_test)

    return data_train, data_test


classes = [0,1,2,3]
classes = random.sample(classes, 4)


(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train / 4
x_test = x_test / 4

x_train = x_train#[0:50, :, :]
x_test = x_test#[0:10, :, :]

y_train = y_train#[0:50]
y_test = y_test#[0:10]

#Задаем параметры нейронов, их модель, параметры STDP и модель STDP#

In [ ]:
!pip install brian2

In [ ]:
from tensorflow.keras.datasets import mnist
from brian2 import *
import brian2.numpy_ as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import time
import random

In [ ]:
n_input = 13*13 #количество входных нейронов
n_output = 4 #количество выходных нейронов

"""
Задаем параметры модели возбуждающих нейронов
"""

v_rest_e = -60.*mV #потенциал покоя  - нейрон стремится к нему без внешнего возбуждения
v_reset_e = -65.*mV #потенциал сброса  - потенцаил нейрона сбрасывается до него после испускания спайка
v_thresh_e = -60.*mV #потенциал порога генерации  - уровень при потенциала, при котором нейрон генерирует свой спайк
tau_ = 100*ms #определяет скорость реакции на входные изменения, а так же насколько быстро нейрон возвращается к своему начальному состоянию после спайка
tau_ = 150*ms # Более длительная временная константа в контексте адаптации порогового потенциала означает, что нейрон медленно адаптируется к изменениям во входной активности. Это может быть полезно для предотвращения чрезмерной реактивности нейрона на кратковременные или незначительные изменения во входных сигналах.


"""
Задаем параметры STDP
"""

lr1 = 1 #коэффициент определяющий скорость обучения в модели STDP - насколько увеличивается вес, если первым активируется пресинаптический нейрон
lr2 = 1 #коэффициент определяющий скорость обучения в модели STDP - насколько уменьшается вес, если первым активируется постсинаптический нейрон
taupre = 20*ms #определяет время, в течение которого влияние пресинаптического спайка на синаптический вес сохраняется после его произошествия.
taupost = taupre # определяет время, в течение которого влияние пресинаптического спайка на синаптический вес сохраняется после его произошествия.
gmax = .05 #максимальное значение, которое может достичь синаптическая связь
dApre = .01 #отражает вклад пресинаптчиеского спайка
dApost = -dApre * taupre / taupost * 1.05
dApost *= gmax #масштабирование треккеров синаптческого веса, чтобы они изменялись соответственно максимальному значению веса
dApre *= gmax #масштабирование треккеров синаптческого веса, чтобы они изменялись соответственно максимальному значению веса


"""
Задаем модель возбуждающего нейрона
"""

neuron_e = '''
            dv/dt = (ge*(0*mV-v) + (v_rest_e-v)) / (100*ms) : volt
            dvt/dt = (v_thresh_e-vt)/(150*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            '''

"""
Задаем адаптивный порог
"""

reset_e = '''
        v = v_reset_e
        vt += 3*mV
        '''

"""
Задаем модель STDP
"""

stdp='''w : 1
    lr1 : 1
    lr2 : 1
    dApre/dt = -Apre / taupre : 1 (event-driven)
    dApost/dt = -Apost / taupost : 1 (event-driven)'''
pre='''ge += w
    Apre += dApre
    w = clip(w + lr1*Apost, 0, gmax)'''
post='''Apost += dApost
    w = clip(w + lr2*Apre, 0, gmax)'''

#Создаем класс модели#

В данной модели используется концепция подгрупп нейронов, где каждая подгруппа возбуждающих нейронов ассоциирована с определенным классом. Это означает, что во время обучения активность нейронов модулируется в зависимости от того, соответствует ли подгруппа текущему целевому классу обрабатываемого образца

In [ ]:
class Model():

  def __init__(self, debug = False):

    app = {}

    app['PG'] = PoissonGroup(n_input, rates=np.zeros(n_input)*Hz, name='PG') #создаем входные Пуассоновские нейроны, пропорциональные яркости изображения

    app['in_ne_1'] = NeuronGroup(n_input, neuron_e, threshold='v>vt', refractory=5*ms, reset=reset_e, method='euler', name='in_ne') #создаем первый скрытый слой

    app['in_ne_2'] = NeuronGroup(n_input, neuron_e, threshold='v>vt', refractory=5*ms, reset=reset_e, method='euler', name='in_ne') #создаем второй скрытый слой

    app['out_ne'] = NeuronGroup(n_output, neuron_e, threshold='v>vt', refractory=5*ms, reset=reset_e, method='euler', name='out_ne') #создадим выходные нейроны


    """
    Соеденяем Пуассоновские нейроны с выходными нейронами
    """

    for cl in range(n_output):

            """
            Соединяем синапсами входные Пуассоновские нейроны и первый скрытый слой
            """
            app[f'S1_{cl}'] = Synapses(app['PG'], app['in_ne_1'][cl:cl+1], stdp, on_pre=pre, on_post=post, method='euler', name=f'S1_{cl}') #создаем синапсы между входными нейронами и выходными нейронами
            app[f'S1_{cl}'].connect() #соеденяем нейроны синапсами
            app[f'S1_{cl}'].w = 'rand()*gmax' #инициализируем случайные веса
            app[f'S1_{cl}'].lr1 = 1 #скорость обучения
            app[f'S1_{cl}'].lr2 = 1 #скорость обучения

            """
            Соединяем первый скрытый слой и второй скрытый слой
            """
            app[f'S2_{cl}'] = Synapses(app['in_ne_1'], app['in_ne_2'][cl:cl+1], stdp, on_pre=pre, on_post=post, method='euler', name=f'S2_{cl}') #создаем синапсы между входными нейронами и выходными нейронами
            app[f'S2_{cl}'].connect() #соеденяем нейроны синапсами
            app[f'S2_{cl}'].w = 'rand()*gmax' #инициализируем случайные веса
            app[f'S2_{cl}'].lr1 = 1 #скорость обучения
            app[f'S2_{cl}'].lr2 = 1 #скорость обучения

            """
            Соединяем синапсами скрытый слой и выходные нейроны
            """
            app[f'S3_{cl}'] = Synapses(app['in_ne_2'], app['out_ne'][cl:cl+1], stdp, on_pre=pre, on_post=post, method='euler', name=f'S3_{cl}') #создаем синапсы между входными нейронами и выходными нейронами
            app[f'S3_{cl}'].connect() #соеденяем нейроны синапсами
            app[f'S3_{cl}'].w = 'rand()*gmax' #инициализируем случайные веса
            app[f'S3_{cl}'].lr1 = 1 #скорость обучения
            app[f'S3_{cl}'].lr2 = 1 #скорость обучения


    #создаем свою сеть в brian2
    self.net = Network(app.values())
    self.net.run(0*second)

    """
    Вспомогательная функция, чтобы удобней было обращаться к отдельным структурам модели по имени обьекта класса
    """

    def __getitem__(self, key):
        return self.net[key]

    """
    Функция для тренировки SNN

    Входные данные:

    X - тренировачная часть датасета
    targets - метки тренировочной части датасета
    epochs - количество эпох для обучения
    """

  def train(self, X, targets, epochs):

    for ep in range(epochs):
      for idx in range(len(X)):

        yt = int(targets[idx]) #ground truth

        for cl in range (len(classes)):

          if cl == classes.index(yt):

            self.net[f'S3_{cl}'].lr1 = 1 #STDP - устанавливаем параметры скорости обучения
            self.net[f'S3_{cl}'].lr2 = 1 #STDP - устанавливаем параметры скорости обучения

          else:

            self.net[f'S3_{cl}'].lr1 = -1 #STDP - устанавливаем параметры скорости обучения
            self.net[f'S3_{cl}'].lr2 = -1 #STDP - устанавливаем параметры скорости обучения

          self.net['PG'].rates = X[idx].ravel()*Hz #устанавливаем Пуассоновские нейроны на нужное значение
          self.net.run(0.140*second) #запускаем численный эксперимент на определенное время

          self.net['PG'].rates = np.zeros(n_input)*Hz #переводим сеть в пассивное состояние - все Пуассоновские нейроны перестают генерировать спайки
          self.net.run(0.15*second) #запускаем численный эксперимент на короткое время, чтобы стабилизировать сеть



    print("train_is_done")


  """
  Функция оценки точности спайковой нейронной сети

  Входные данные:

  X - тестовая часть датасета
  targets - метки тестовой части датасета
  """

  def evaluate(self, X, targets):

    for cl in range(len(classes)):

      """
      Здесь встает вопрос: нужно ли отключать все STDP???? - у дедов все остальные веса фиксированы и константы, поэтому вопрос остается открытым
      """

      self.net[f'S1_{cl}'].lr1 = 0 #выключаем STDP
      self.net[f'S1_{cl}'].lr2 = 0 #выключаем STDP

      self.net[f'S2_{cl}'].lr1 = 0 #выключаем STDP
      self.net[f'S2_{cl}'].lr2 = 0 #выключаем STDP

      self.net[f'S3_{cl}'].lr1 = 0 #выключаем STDP
      self.net[f'S3_{cl}'].lr2 = 0 #выключаем STDP

    true_ans = 0 #количество правильно классифицированных картинок

    for idx in range(len(X)):

      print(f"image in test:{len(X)}")

      yt = int(targets[idx]) #забираем правильные метки

      #предсказание SNN определяем по нейрону с наибольшим количеством выходных спайков
      mon = SpikeMonitor(self.net['out_ne'], record=True, name='RM')
      self.net.add(mon)
       #добавляем монитор
      # active mode
      self.net['PG'].rates = X[idx].ravel()*Hz
      self.net.run(0.35*second)

      print(f"moncount:{mon.count} for {yt} - image number: {idx}")

      counts = [] #массив спайков выходных нейронов

      for cl in range(len(classes)):
        counts.append(sum(mon.count[cl:cl+1]))

      if counts.index(max(counts)) == classes.index(yt):
        true_ans+=1


      #возвразаем систему в пассивный режим для стабильности

      self.net['PG'].rates = np.zeros(n_input)*Hz
      self.net.run(0.15*second)

      self.net.remove(self.net['RM']) #освобождаем память от монитора

    return true_ans/len(X)

#Основной процесс - обучение и тестирование модели#

In [ ]:
"""
Функция, которая реализует процесс обучения и тестирования модели
Входные данные:

train_items - количество тренировочных картинок
test_items - количество тестовых картинок

"""

def train_test(x_train, x_test, y_train, y_test):

  model = Model()
  model.train(x_train, y_train, 10)

  accuracy = model.evaluate(x_test, y_test)

  print(f"Accuracy of SNN is:{accuracy}")


start = time.time()
train_test(x_train, x_test, y_train, y_test)
end = time.time()
print('SNN time:', end - start)

WARNING    'lr1' is an internal variable of group 'S1_0', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'lr1' is an internal variable of group 'S1_1', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'lr1' is an internal variable of group 'S1_2', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'lr1' is an internal variable of group 'S1_3', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'lr2' is an internal variable of group 'S1_0', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.reso

train_is_done
image in test:118
moncount:<RM.count: array([2, 2, 3, 2], dtype=int32)> for 3 - image number: 0
image in test:118
moncount:<RM.count: array([3, 3, 2, 3], dtype=int32)> for 2 - image number: 1
image in test:118
moncount:<RM.count: array([3, 4, 4, 4], dtype=int32)> for 1 - image number: 2
image in test:118
moncount:<RM.count: array([4, 4, 5, 4], dtype=int32)> for 1 - image number: 3
image in test:118
moncount:<RM.count: array([3, 3, 3, 3], dtype=int32)> for 1 - image number: 4
image in test:118
moncount:<RM.count: array([3, 3, 3, 3], dtype=int32)> for 2 - image number: 5
image in test:118
moncount:<RM.count: array([1, 2, 2, 1], dtype=int32)> for 1 - image number: 6
image in test:118
moncount:<RM.count: array([4, 4, 4, 4], dtype=int32)> for 0 - image number: 7
image in test:118
moncount:<RM.count: array([3, 3, 3, 3], dtype=int32)> for 3 - image number: 8
image in test:118
moncount:<RM.count: array([2, 2, 2, 2], dtype=int32)> for 1 - image number: 9
image in test:118
moncount